<b>Face Recognition</b>
--

In [2]:
!pip install mtcnn

     |████████████████████████████████| 2.3 MB 4.4 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
from keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from numpy import * 
import pandas as pd 

In [4]:
model = load_model('../input/facenet-keras/facenet_keras.h5')      

model.inputs

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]

In [5]:
model.outputs

[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]

In [6]:
def get_embedding(model, face_pixels): 
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

def extract_face(filename, required_size=(160, 160)): 
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

In [7]:
originalface = extract_face('./__results___13_1.png')
testface     = extract_face('./__results___13_1.png')

In [9]:
originalembedding = get_embedding(model,originalface)    
testembedding = get_embedding(model,testface)

In [10]:
dist = linalg.norm(testembedding-originalembedding)    
dist

0.0

If 0.0, that means same!